In [ ]:
import pandas as pd
from google.colab import drive
import os

drive.mount('/content/drive')
Drive = "/content/drive/My Drive/data_analysis/competition/Signate_sony_pm2.5_liner"

Input = os.path.join(Drive, 'input')
Output = os.path.join(Drive, 'output')
Submission = os.path.join(Drive, "submission")

In [ ]:
train = pd.read_csv(os.path.join(Input, 'train.csv')) # 学習用データ
test = pd.read_csv(os.path.join(Input, 'test.csv')) # 学習用データ
submit = pd.read_csv(os.path.join(Input, 'submit_sample.csv'),header=None) # 応募用サンプルファイル

In [ ]:
test.shape
test

In [ ]:
train.shape

In [ ]:
submit

In [ ]:
#データ解析用ライブラリ
import pandas as pd
import numpy as np

#データ可視化ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns

#LightGBMライブラリ
import lightgbm as lgb

#訓練データとモデル評価用データに分けるライブラリ
from sklearn.model_selection import train_test_split

In [ ]:
fig, ax = plt.subplots(figsize=(50, 24)) 
train_num = train.drop(["year","month","Country","City","day"], axis=1)
# sns.heatmap(train_num.corr(), square=True, vmax=1, vmin=-1, center=0, annot=True)

In [ ]:
# X = train[["year","lon","co_mid","co_cnt","co_max","o3_max","o3_var","Country"]]
X = train.drop(["id","pm25_mid"],axis=1)
X = pd.get_dummies(X)

test = test.drop(["id"],axis=1)
test = pd.get_dummies(test)

object1 = {"2019":0, "2020":1, "2021":2}

X["year"] = X["year"].map(object1)
test["year"] = test["year"].map(object1)

y = train["pm25_mid"]

In [ ]:
from sklearn.model_selection import KFold

model = []

def fit_lgb(x, y, model_params, fit_params, fobj=None, feval=None):
  fold = 0
  kf = KFold(n_splits=5, shuffle=True, random_state=42)

  for fold, (train_indices, valid_indices) in enumerate(kf.split(X)):
    fold += 1
    X_train, X_test = X.iloc[train_indices], X.iloc[valid_indices]
    y_train, y_test = y[train_indices], y[valid_indices]

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test)

    lgb_model = lgb.train(model_params,
                       train_set=lgb_train,
                       valid_sets=[lgb_train, lgb_eval],
                       fobj=fobj,
                       feval=feval,
                       verbose_eval=fit_params['verbose_eval'],
                       num_boost_round=fit_params['num_boost_rounds'],
                       callbacks=[lgb.early_stopping(fit_params['early_stopping_rounds'])],
                       #early_stopping_rounds=fit_params['early_stopping_rounds']
                       )
    pred_valid = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)
    model.append(lgb_model)
    print(f' - fold{fold}_RMSE - {lgb_model}')
    
  return model

In [ ]:
lgb_fit_params ={'num_boost_rounds': 100000,
                 'early_stopping_rounds': 000,
                 'verbose_eval': 100,
                }


lgb_model_params = {'boosting_type': 'gbdt',
                
                    'objective': 'rmse',

                    'learning_rate': 0.05,
                    'max_depth': -1,
                    'num_leaves': 71,
                    'min_data_in_leaf': 20,
                    'max_bin': 255,
                    
                    'reg_lambda': 1.0,
                    'reg_alpha': 1.,
                   
                    'colsample_bytree': 0.8,
                    'subsample': 0.8,
                    'subsample_freq': 1,
                    'verbose': -1,
                    'n_jobs': -1,
                    }

In [ ]:
getdata = fit_lgb(X, y, lgb_model_params,lgb_fit_params)

In [ ]:
preds = np.zeros(len(test))

# 各モデルで推論
for model in models:
    pred = model.predict(test, num_iteration=model.best_iteration)
    predict += pred / len(models)

In [ ]:
predict_train = predict_test(getdata, X)
predict_train

In [ ]:
predict_train.shape

In [ ]:
predict.shape

In [ ]:
#関数の処理で必要なライブラリ
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
 
#予測値と正解値を描写する関数
def True_Pred_map(pred_df,true_df):
    RMSE = np.sqrt(mean_squared_error(pred_df, true_df))
    R2 = r2_score(pred_df, true_df) 
    plt.figure(figsize=(8,8))
    ax = plt.subplot(111)
    ax.scatter('true', 'pred', data=pred_df)
    ax.set_xlabel('True Value', fontsize=15)
    ax.set_ylabel('Pred Value', fontsize=15)
    ax.set_xlim(pred_df.min().min()-0.1 , pred_df.max().max()+0.1)
    ax.set_ylim(pred_df.min().min()-0.1 , pred_df.max().max()+0.1)
    x = np.linspace(pred_df.min().min()-0.1, pred_df.max().max()+0.1, 2)
    y = x
    ax.plot(x,y,'r-')
    plt.text(0.1, 0.9, 'RMSE = {}'.format(str(round(RMSE, 5))), transform=ax.transAxes, fontsize=15)
    plt.text(0.1, 0.8, 'R^2 = {}'.format(str(round(R2, 5))), transform=ax.transAxes, fontsize=15)

In [ ]:
True_Pred_map(predict_train,y)

In [ ]:
pred_df = pd.concat([y.reset_index(drop=True), pd.Series(predict)], axis=1)
pred_df.columns = ['true', 'pred']

In [ ]:
lgb.plot_importance(getdata[1], height=0.5, figsize=(8,32))

In [ ]:
submit_data = predict
submit_data.shape

In [ ]:
submit.shape

In [ ]:
submit.iloc[:, 1] = submit_data
submit

In [ ]:
from google.colab import drive, files
submit.to_csv(os.path.join(Submission, 'submission.csv'),index=False,header=None)
files.download(os.path.join(Submission, 'submission.csv'))